In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

TARGET_VARIABLE = 'Traffic Subtype'
DROP_COLUMNS = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp']
TARGET_TO_DROP = {'Label': ['Traffic Type', 'Traffic Subtype'],
                  'Traffic Type': ['Label', 'Traffic Subtype'],
                  'Traffic Subtype': ['Label', 'Traffic Type']}


# Load extracted bi. flow features
print('Loading dataset... This may take some time. Please monitor the RAM usage at the top right corner.')
df = pd.read_csv('data.csv')

# Initial data overview
counts = df.groupby(['Label', 'Traffic Type', 'Traffic Subtype']).size().reset_index(name='Counts')
print(counts)

Loading dataset... This may take some time. Please monitor the RAM usage at the top right corner.
        Label           Traffic Type        Traffic Subtype   Counts
0      Benign                  Audio                  Audio      190
1      Benign             Background             Background       32
2      Benign                   Text                   Text      209
3      Benign                  Video             Video HTTP      376
4      Benign                  Video              Video RTP      349
5      Benign                  Video              Video UDP      145
6   Malicious             Bruteforce         Bruteforce DNS    22179
7   Malicious             Bruteforce         Bruteforce FTP     3485
8   Malicious             Bruteforce        Bruteforce HTTP      628
9   Malicious             Bruteforce         Bruteforce SSH     3967
10  Malicious             Bruteforce      Bruteforce Telnet     4913
11  Malicious                    DoS                DoS ACK   936307
12  M

In [2]:
# Drop 5-tuple collumns and timestamp
df = df.drop(columns=DROP_COLUMNS)

# Filter out duplicates within the same target
df = df.round(3)
df = df.drop_duplicates()
df = df.drop(columns=TARGET_TO_DROP[TARGET_VARIABLE])

# Separate features and target
X = df.drop(TARGET_VARIABLE, axis=1)
y = df[TARGET_VARIABLE]



# Encode target
le = LabelEncoder()
y = le.fit_transform(y)

# Compute train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42) 

In [3]:
# Identifying Numerical and Categorical columns
numerical_cols = X_train.select_dtypes(include=[np.number]).columns.to_list()
numerical_cols.remove('Protocol')
categorical_cols = X_train.select_dtypes(include=[object]).columns.to_list()
categorical_cols.append('Protocol')

# Pipelines for Numerical and Categorical Data Transformations
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Column Transformer combining both pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Apply preprocessor to train and test data
preprocessor.fit(X_train)
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [4]:
model = RandomForestClassifier(n_estimators=100, random_state=42, verbose=0, n_jobs=-1)
model.fit(X_train, y_train)

# Predicting Test Data
y_pred = model.predict(X_test)

# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred)) 

Accuracy: 0.9973414798773605
F1 Score: 0.9250302744252419
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94        38
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00      4295
           3       1.00      0.99      1.00       697
           4       1.00      0.99      1.00       126
           5       1.00      0.99      1.00       785
           6       0.96      0.96      0.96       974
           7       0.99      0.99      0.99    157145
           8       1.00      1.00      1.00    141110
           9       1.00      1.00      1.00    141580
          10       0.99      0.99      0.99    130706
          11       0.99      0.99      0.99     15479
          12       1.00      1.00      1.00         2
          13       0.86      1.00      0.92         6
          14       0.99      0.99      0.99    155225
          15       1.00      1.00      1.00    194670